In [1]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.pool import NullPool
import matplotlib as plt
import matplotlib.pylab as plt
import seaborn as sns
from tqdm import tqdm
from PIL import Image
import requests
from io import BytesIO


#### 2020년 11월 27일 ~ 12월 6일 상점업 신청

In [164]:
q = """
SELECT date_format(create_at, '%Y-%m-%d') AS date,
       uid,
       count(b.pid) AS p_cnt
FROM ad_super_up_shop a
JOIN ad_super_up_shop_products b ON a.sus_id = b.sus_id
WHERE create_at BETWEEN '2020-11-27 00:00:00' AND '2020-12-06 23:59:59'
  AND status < 3
GROUP BY 1, 2
"""
shop_up = pd.read_sql(q, con=mysql)

In [165]:
shop_up['uid'].nunique()

1860

#### 2020년 11월 27일 ~ 12월 6일 슈퍼업 신청

In [166]:
q = """
SELECT date_format(create_at, "%Y-%m-%d") AS date, uid, pid
FROM ad_super_up
WHERE create_at BETWEEN '2020-11-27 00:00:00' AND '2020-12-06 23:59:59'
    AND status < 3
"""
super_up = pd.read_sql(q, con=mysql)

In [167]:
super_up['uid'].nunique()

850

In [168]:
len(set(shop_up['uid'].tolist() + super_up['uid'].tolist()))

2601

2020년 11월 26일 ~ 12월 2일 (일주일) 동안  
- 상점업 신청 유저 수 : 1860
- 슈퍼업 신청 유저 수 : 850 
- 상점업 & 슈퍼업 신청 유저 수 : 109  
- 둘 중 하나라도 신청한 유저 수 : 2601

### 신규 광고 SA 사용 현황

In [173]:
q = """
SELECT a.uid
FROM service1_quicket.ad_set a
WHERE a.uid IN
    (SELECT UID
     FROM service1_quicket.ad_super_up_shop
     WHERE create_at BETWEEN '2020-11-27 00:00:00' AND '2020-12-06 23:59:59'
       AND status < 3
     GROUP BY 1
     UNION SELECT UID
     FROM service1_quicket.ad_super_up
     WHERE create_at BETWEEN '2020-11-27 00:00:00' AND '2020-12-06 23:59:59'
       AND status < 3
     GROUP BY 1)
     AND status != 2
GROUP BY 1
"""

sa_uid= pd.read_sql(q, con=bun_dw)

In [174]:
shop_up['uid'] = shop_up['uid'].astype(str)
super_up['uid'] = super_up['uid'].astype(str)
sa_uid['uid'] = sa_uid['uid'].astype(str)

In [183]:
sa_uid[sa_uid['uid'].isin(shop_up['uid'])]['uid'].nunique()

546

In [184]:
sa_uid[sa_uid['uid'].isin(super_up['uid'])]['uid'].nunique()

572

In [185]:
sa_uid['uid'].nunique()

1037

2월 4일 15시 기준까지  
2020년 11월 26일 ~ 12월 2일 (일주일) 동안 상점업 & 슈퍼업을 신청했던 유저들(2,601명) 중  
        sa광고를 신청한 이력이 있는 유저 수 : 1,037 = 39.8%
- 상점업 사용 유저(1,860) 중: 546 = 29.4%
- 슈퍼업 사용 유저(850) 중: 572 = 67.3%

### 10.12 - 12.06 두달간 슈퍼업 & 상점업 사용자들의 SA 신청 상품 수 비교 (12.07-01.31 두달간)

In [186]:
q = """
SELECT a.UID,
       p.pid
FROM service1_quicket.ad_set a
JOIN service1_quicket.ad_set_product p ON a.id = p.ad_set_id
WHERE a.uid IN
    (SELECT UID
     FROM service1_quicket.ad_super_up
     WHERE create_at BETWEEN '2020-10-12 00:00:00' AND '2020-12-06 23:59:59'
       AND status < 3
     GROUP BY 1
     UNION
     SELECT UID
     FROM service1_quicket.ad_super_up_shop
     WHERE create_at BETWEEN '2020-10-12 00:00:00' AND '2020-12-06 23:59:59'
       AND status < 3
     GROUP BY 1)
     AND a.created_at BETWEEN '2020-12-07 00:00:00' AND '2021-01-31 23:59:59'
GROUP BY 1, 2
"""

su_and_sa = pd.read_sql(q, con=bun_dw)

In [187]:
q = """
SELECT uid, pid
FROM service1_quicket.ad_super_up
WHERE create_at BETWEEN '2020-10-12 00:00:00' AND '2020-12-06 23:59:59'
  AND status < 3
GROUP BY 1, 2
"""
su = pd.read_sql(q, con=bun_dw)

In [189]:
q = """
SELECT a.uid, b.pid
FROM service1_quicket.ad_super_up_shop a 
JOIN service1_quicket.ad_super_up_shop_products b ON a.sus_id = b.sus_id
WHERE a.create_at BETWEEN '2020-10-12 00:00:00' AND '2020-12-06 23:59:59'
    AND a.status < 3
GROUP BY 1, 2
"""
shop = pd.read_sql(q, con=bun_dw)

In [203]:
su_and_sa_product_cnt = su_and_sa.groupby('uid', as_index=False)['pid'].count()
su_product_cnt = su.groupby('uid', as_index=False)['pid'].count()
shop_product_cnt = shop.groupby('uid', as_index=False)['pid'].count()

In [204]:
su_and_sa_product_cnt.columns = ['uid', 'sa_p_cnt']
su_product_cnt.columns = ['uid', 'su_p_cnt']
shop_product_cnt.columns = ['uid', 'shop_p_cnt']

In [205]:
su_and_sa_product_cnt['uid'] = su_and_sa_product_cnt['uid'].astype(str)
su_product_cnt['uid'] = su_product_cnt['uid'].astype(str)
shop_product_cnt['uid'] = shop_product_cnt['uid'].astype(str)
su_vs_sa_product_cnt = pd.merge(su_and_sa_product_cnt, su_product_cnt, on='uid', how='outer')
su_vs_sa_product_cnt = pd.merge(su_vs_sa_product_cnt, shop_product_cnt, on='uid', how='outer')

In [207]:
su_vs_sa_product_cnt.to_csv('su_vs_sa_product_cnt.csv')

In [208]:
su_vs_sa_product_cnt.head()

,uid,sa_p_cnt,su_p_cnt,shop_p_cnt
0,7379,21.0,NaN,9.0
1,13170,2.0,NaN,11.0
2,14279,4.0,NaN,4.0
3,14552,3.0,NaN,7.0
4,14692,4.0,6.0,NaN


In [209]:
# 슈퍼업이나 상점업은 사용했지만 신규 광고는 사용하지 않은 상점 수
su_vs_sa_product_cnt[su_vs_sa_product_cnt['sa_p_cnt'].isna()]['uid'].nunique()

9014

In [211]:
# 슈퍼업이나 상점업 사용했지만 SA는 사용하지 않은 유저들의 슈퍼업 신청 상품 수
su_vs_sa_product_cnt[su_vs_sa_product_cnt['sa_p_cnt'].isna()][['su_p_cnt', 'shop_p_cnt']].describe()

,su_p_cnt,shop_p_cnt
count,3104.000000,6569.000000
mean,2.272229,13.742883
std,4.633558,67.987204
min,1.000000,1.000000
25%,1.000000,2.000000
50%,1.000000,5.000000
75%,3.000000,11.000000
max,180.000000,2398.000000


In [214]:
#SA도 사용하는 유저들의 SA 신청 상품 수
su_vs_sa_product_cnt[~su_vs_sa_product_cnt['sa_p_cnt'].isna()][['sa_p_cnt', 'su_p_cnt', 'shop_p_cnt']].describe()

,sa_p_cnt,su_p_cnt,shop_p_cnt
count,2480.000000,1350.000000,1648.000000
mean,14.332258,8.690370,59.853762
std,40.229646,21.390481,153.525582
min,1.000000,1.000000,1.000000
25%,2.000000,1.000000,10.000000
50%,5.000000,3.000000,18.000000
75%,11.000000,6.000000,42.000000
max,1043.000000,297.000000,1894.000000


In [215]:
only_su_uid = su_vs_sa_product_cnt[su_vs_sa_product_cnt['sa_p_cnt'].isna()]['uid'].unique()
only_su_uids = ','.join('\'' + str(i) + '\'' for i in only_su_uid)
q = f'''
SELECT u.id AS uid,
       CASE
           WHEN u.status = 0 THEN 'ing'
           WHEN u.status = 1 THEN 'dead'
           ELSE 'dead'
       END AS status,
       CASE
           WHEN t.policy_id = 1 THEN 'normal'
           WHEN t.policy_id = 2 THEN 'block'
           ELSE 'block'
       END AS policy
FROM user u
LEFT JOIN user_auth_token t ON u.id = t.uid
WHERE u.id IN ({only_su_uids})
'''
only_su_uid_status = pd.read_sql(q, con=mysql)

In [216]:
only_su_uid_status

,uid,status,policy
0,4148,ing,normal
1,8890,ing,normal
2,10067,ing,normal
3,11281,ing,normal
4,12548,ing,normal
...,...,...,...
9033,74540104,ing,normal
9034,74541438,dead,block
9035,74542066,dead,block
9036,74544695,ing,normal


In [217]:
only_su_uid_status.groupby(['status', 'policy']).count()

uid
status policy      
dead   block   1007
ing    block    376
       normal  7655

In [268]:
# 슈퍼업, 상점업 사용했던 유저들 중 sa를 사용하지 않고 차단당하거나 탈퇴하지 않는 live 유저들의 up 사용이력
live_uids = ','.join('\''+ str(i) +'\'' for i in only_su_uid_status[only_su_uid_status['policy'] == 'normal']['uid'].unique())

q = f"""
SELECT uid, pid, up_type, source_type
FROM up_count_history
WHERE register_at between '2020-12-07 00:00:00' AND '2021-01-31 23:59:59'
    AND uid IN ({live_uids})
"""
live_uids_up = pd.read_sql(q, con=mysql)

In [218]:
only_su_uid_status['uid'] = only_su_uid_status['uid'].astype(str)

only_su_uid_status = pd.merge(only_su_uid_status, su_vs_sa_product_cnt[su_vs_sa_product_cnt['sa_p_cnt'].isna()], on='uid')

In [221]:
only_su_uid_status[(only_su_uid_status['policy'] == 'block')][['su_p_cnt', 'shop_p_cnt']].describe()

,su_p_cnt,shop_p_cnt
count,556.000000,1028.000000
mean,3.715827,26.937743
std,9.381705,116.628514
min,1.000000,1.000000
25%,1.000000,3.000000
50%,3.000000,8.000000
75%,3.000000,13.000000
max,180.000000,1938.000000


In [222]:
only_su_uid_status[(only_su_uid_status['status'] == 'ing') & (only_su_uid_status['policy'] == 'normal')][['su_p_cnt', 'shop_p_cnt']].describe()

,su_p_cnt,shop_p_cnt
count,2554.000000,5560.000000
mean,1.956539,11.279496
std,2.531755,53.946696
min,1.000000,1.000000
25%,1.000000,2.000000
50%,1.000000,5.000000
75%,3.000000,11.000000
max,48.000000,2398.000000


In [264]:
only_su_uid_status

,uid,status,policy,sa_p_cnt,su_p_cnt,shop_p_cnt
0,4148,ing,normal,NaN,1.0,NaN
1,8890,ing,normal,NaN,1.0,NaN
2,10067,ing,normal,NaN,4.0,NaN
3,11281,ing,normal,NaN,1.0,NaN
4,12548,ing,normal,NaN,3.0,NaN
...,...,...,...,...,...,...
9033,74540104,ing,normal,NaN,NaN,1.0
9034,74541438,dead,block,NaN,NaN,2.0
9035,74542066,dead,block,NaN,NaN,7.0
9036,74544695,ing,normal,NaN,NaN,1.0


In [223]:
only_su_uid_status[(only_su_uid_status['status'] == 'ing') & (only_su_uid_status['policy'] == 'normal')]['uid']

0           4148
1           8890
2          10067
3          11281
4          12548
          ...   
9030    74530294
9032    74531804
9033    74540104
9036    74544695
9037    74547057
Name: uid, Length: 7655, dtype: object

### 광고 소진 비교

In [252]:
q = """
SELECT a.UID,
       sum(b.paid_point) + sum(b.free_point) as point
FROM service1_quicket.ad_set a
JOIN service1_quicket.ad_set_daily_budget b ON a.id = b.ad_set_id
WHERE a.uid IN
    (SELECT UID
     FROM service1_quicket.ad_super_up
     WHERE create_at BETWEEN '2020-10-02 00:00:00' AND '2020-12-06 23:59:59'
       AND status < 3
     GROUP BY 1
     UNION
     SELECT UID
     FROM service1_quicket.ad_super_up_shop
     WHERE create_at BETWEEN '2020-10-02 00:00:00' AND '2020-12-06 23:59:59'
         AND status < 3)
     AND a.created_at BETWEEN '2020-12-07 00:00:00' AND '2021-01-31 23:59:59'
GROUP BY 1
"""
sa_point = pd.read_sql(q, con=bun_dw)

In [253]:
q = """
SELECT uid, sum(pay_sum) as point
FROM service1_quicket.ad_super_up a
JOIN service1_quicket.ad_super_up_point b ON a.suid = b.suid
WHERE a.create_at BETWEEN '2020-10-05 00:00:00' AND '2020-11-29 23:59:59'
  AND a.status < 3
GROUP BY 1
"""
su_point = pd.read_sql(q, con=bun_dw)

In [254]:
q = """
SELECT uid, sum(b.pay_sum) as point
FROM service1_quicket.ad_super_up_shop a
JOIN service1_quicket.ad_super_up_shop_point b ON a.sus_id = b.sus_id
WHERE a.create_at BETWEEN '2020-10-05 00:00:00' AND '2020-11-29 23:59:59'
  AND a.status < 3
GROUP BY 1
"""
shop_point = pd.read_sql(q, con=bun_dw)

In [255]:
sa_point['uid'] = sa_point['uid'].astype(str)
su_point['uid'] = su_point['uid'].astype(str)
shop_point['uid'] = shop_point['uid'].astype(str)

sa = pd.merge(su_and_sa_product_cnt, sa_point, on='uid')
su = pd.merge(su_product_cnt, su_point, on='uid')
shop = pd.merge(shop_product_cnt, shop_point, on='uid')

In [256]:
sa.columns = ['uid', 'sa_p_cnt', 'sa_point']
su.columns = ['uid', 'su_p_cnt', 'su_point']
shop.columns = ['uid', 'shop_p_cnt', 'shop_point']
sa['sa_point_per_product'] = sa['sa_point'] / sa['sa_p_cnt']
su['su_point_per_product'] = su['su_point'] / su['su_p_cnt']

In [257]:
su_vs_sa_point = pd.merge(sa, su, on='uid', how='outer')
su_vs_sa_point = pd.merge(su_vs_sa_point, shop, on='uid', how='outer')

In [234]:
su_vs_sa_point.to_csv('su_vs_sa_point.csv')

In [259]:
su_vs_sa_point.count()

uid                     10539
sa_p_cnt                 2480
sa_point                 2480
sa_point_per_product     2480
su_p_cnt                 4246
su_point                 4246
su_point_per_product     4246
shop_p_cnt               7203
shop_point               7203
dtype: int64

In [261]:
su_vs_sa_point.describe()

,sa_p_cnt,sa_point,sa_point_per_product,su_p_cnt,su_point,su_point_per_product,shop_p_cnt,shop_point
count,2480.000000,2.480000e+03,2.480000e+03,4246.000000,4.246000e+03,4246.000000,7203.000000,7.203000e+03
mean,14.332258,1.639060e+05,1.706516e+04,4.328309,9.393974e+04,17015.442157,25.216576,3.069819e+04
std,40.229646,6.897332e+05,6.009448e+04,13.025935,4.535565e+05,32269.501092,99.316094,1.045650e+05
min,1.000000,0.000000e+00,0.000000e+00,1.000000,0.000000e+00,0.000000,1.000000,0.000000e+00
25%,2.000000,8.971000e+03,2.579167e+03,1.000000,2.800000e+03,2660.000000,3.000000,0.000000e+00
50%,5.000000,3.889250e+04,6.669385e+03,2.000000,5.600000e+03,2940.000000,8.000000,0.000000e+00
75%,11.000000,1.051450e+05,1.444401e+04,3.000000,3.969000e+04,13738.375000,15.000000,0.000000e+00
max,1043.000000,1.885945e+07,1.245494e+06,297.000000,1.520380e+07,366520.000000,2398.000000,1.131144e+06


In [262]:
su_vs_sa_point[~su_vs_sa_point['sa_point'].isna()].describe()

,sa_p_cnt,sa_point,sa_point_per_product,su_p_cnt,su_point,su_point_per_product,shop_p_cnt,shop_point
count,2480.000000,2.480000e+03,2.480000e+03,1262.000000,1.262000e+03,1262.000000,1498.000000,1.498000e+03
mean,14.332258,1.639060e+05,1.706516e+04,9.129160,2.549440e+05,37635.033545,64.548064,1.043322e+05
std,40.229646,6.897332e+05,6.009448e+04,22.040478,7.806464e+05,45821.835592,160.209639,1.778808e+05
min,1.000000,0.000000e+00,0.000000e+00,1.000000,0.000000e+00,0.000000,1.000000,0.000000e+00
25%,2.000000,8.971000e+03,2.579167e+03,1.000000,1.176000e+04,5053.125000,10.000000,0.000000e+00
50%,5.000000,3.889250e+04,6.669385e+03,3.000000,6.750450e+04,17936.173969,20.000000,3.700000e+04
75%,11.000000,1.051450e+05,1.444401e+04,6.000000,1.759432e+05,54896.486842,46.000000,8.969900e+04
max,1043.000000,1.885945e+07,1.245494e+06,297.000000,1.520380e+07,366520.000000,1894.000000,1.131144e+06


In [263]:
su_vs_sa_point[su_vs_sa_point['sa_point'].isna()].describe()

,sa_p_cnt,sa_point,sa_point_per_product,su_p_cnt,su_point,su_point_per_product,shop_p_cnt,shop_point
count,0.0,0.0,0.0,2984.000000,2.984000e+03,2984.000000,5705.000000,5705.000000
mean,NaN,NaN,NaN,2.297922,2.584745e+04,8294.958132,14.889045,11363.617353
std,NaN,NaN,NaN,4.714801,1.397278e+05,18394.076485,72.144982,60854.381190
min,NaN,NaN,NaN,1.000000,0.000000e+00,0.000000,1.000000,0.000000
25%,NaN,NaN,NaN,1.000000,0.000000e+00,0.000000,2.000000,0.000000
50%,NaN,NaN,NaN,1.000000,2.940000e+03,2940.000000,6.000000,0.000000
75%,NaN,NaN,NaN,3.000000,9.240000e+03,5600.000000,11.000000,0.000000
max,NaN,NaN,NaN,180.000000,3.838380e+06,240240.000000,2398.000000,940800.000000


In [258]:
su_vs_sa_point[(~su_vs_sa_point['sa_point'].isna()) & (~su_vs_sa_point['su_point'].isna())].describe()

,sa_p_cnt,sa_point,sa_point_per_product,su_p_cnt,su_point,su_point_per_product,shop_p_cnt,shop_point
count,1262.000000,1.262000e+03,1.262000e+03,1262.000000,1.262000e+03,1262.000000,471.000000,471.000000
mean,15.792393,2.164698e+05,2.124063e+04,9.129160,2.549440e+05,37635.033545,69.630573,106003.838641
std,41.107194,8.690920e+05,7.019791e+04,22.040478,7.806464e+05,45821.835592,156.100443,177199.583577
min,1.000000,0.000000e+00,0.000000e+00,1.000000,0.000000e+00,0.000000,1.000000,0.000000
25%,2.000000,1.040300e+04,3.003589e+03,1.000000,1.176000e+04,5053.125000,12.000000,0.000000
50%,4.000000,4.378900e+04,7.790250e+03,3.000000,6.750450e+04,17936.173969,22.000000,37000.000000
75%,11.000000,1.282468e+05,1.940256e+04,6.000000,1.759432e+05,54896.486842,52.500000,106096.500000
max,656.000000,1.885945e+07,1.245494e+06,297.000000,1.520380e+07,366520.000000,1620.000000,940800.000000


In [248]:
su_vs_sa_point[(su_vs_sa_point['sa_point'] > su_vs_sa_point['su_point']) & (~su_vs_sa_point['sa_point'].isna()) & (~su_vs_sa_point['su_point'].isna())].describe()

,sa_p_cnt,sa_point,sa_point_per_product,su_p_cnt,su_point,su_point_per_product,shop_p_cnt,shop_point
count,516.000000,5.160000e+02,5.160000e+02,516.000000,5.160000e+02,516.000000,237.000000,237.000000
mean,19.732558,3.597473e+05,3.479274e+04,4.269380,1.053909e+05,25428.494576,72.983122,110240.687764
std,45.574203,1.278815e+06,1.066126e+05,8.287967,3.656945e+05,41095.197036,161.872490,182293.047526
min,1.000000,1.000000e+02,5.000000e+01,1.000000,0.000000e+00,0.000000,1.000000,0.000000
25%,3.000000,2.942150e+04,5.319355e+03,1.000000,3.080000e+03,2940.000000,11.000000,0.000000
50%,6.000000,8.153800e+04,1.053612e+04,2.000000,1.778000e+04,6379.333333,22.000000,37000.000000
75%,15.000000,2.255088e+05,2.510150e+04,3.000000,7.535500e+04,30958.752155,56.000000,104440.000000
max,402.000000,1.885945e+07,1.245494e+06,89.000000,6.029254e+06,366520.000000,1620.000000,833683.000000


In [249]:
su_vs_sa_point[(su_vs_sa_point['sa_point'] < su_vs_sa_point['su_point']) & (~su_vs_sa_point['sa_point'].isna()) & (~su_vs_sa_point['su_point'].isna())].describe()

,sa_p_cnt,sa_point,sa_point_per_product,su_p_cnt,su_point,su_point_per_product,shop_p_cnt,shop_point
count,743.000000,7.430000e+02,743.000000,743.000000,7.430000e+02,743.000000,233.000000,233.000000
mean,13.113055,1.177850e+05,11911.293011,12.531629,3.598353e+05,46264.211483,66.424893,101896.738197
std,37.568168,3.518798e+05,16376.613203,27.380394,9.570858e+05,47033.970168,150.594926,172519.198131
min,1.000000,0.000000e+00,0.000000,1.000000,2.660000e+03,700.000000,1.000000,0.000000
25%,1.000000,5.216500e+03,1854.333333,1.000000,4.021500e+04,9640.277778,12.000000,0.000000
50%,3.000000,2.182500e+04,6000.000000,3.000000,1.121610e+05,28402.017241,21.000000,33291.000000
75%,8.000000,8.965150e+04,15307.642857,8.000000,2.885330e+05,69819.750000,47.000000,107753.000000
max,656.000000,5.411346e+06,124050.000000,297.000000,1.520380e+07,274715.000000,1339.000000,940800.000000


In [108]:
su['su_point'].describe()

count    5.833000e+03
mean     7.047843e+04
std      3.892566e+05
min      0.000000e+00
25%      2.800000e+03
50%      2.940000e+03
75%      1.955100e+04
max      1.520380e+07
Name: su_point, dtype: float64

In [109]:
sa['sa_point'].describe()

count    1.398000e+03
mean     2.158899e+05
std      8.784611e+05
min      0.000000e+00
25%      9.697250e+03
50%      4.163300e+04
75%      1.182398e+05
max      1.885945e+07
Name: sa_point, dtype: float64

In [110]:
su['su_point_per_product'].describe()

count      5833.000000
mean      13113.541829
std       27116.013617
min           0.000000
25%        2660.000000
50%        2940.000000
75%        9310.000000
max      273714.000000
Name: su_point_per_product, dtype: float64

In [111]:
sa['sa_point_per_product'].describe()

count    1.398000e+03
mean     2.106322e+04
std      7.012715e+04
min      0.000000e+00
25%      2.818500e+03
50%      7.494250e+03
75%      1.831310e+04
max      1.245494e+06
Name: sa_point_per_product, dtype: float64